In [ ]:
#!pip install unsloth "xformers==0.0.28.post2"
# Also get the latest nightly Unsloth!
#!pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
!pip install --no-deps unsloth


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.1/138.1 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.5.

In [ ]:
import unsloth
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset
from unsloth import is_bfloat16_supported
from unsloth.chat_templates import train_on_responses_only
from unsloth import FastModel
from unsloth.chat_templates import get_chat_template
from transformers import TrainingArguments, DataCollatorForSeq2Seq

#Refererence:
max_seq_length = 2048 # Supports RoPE Scaling interally, so choose any!
dataset = load_dataset("json", data_files = ["./finetune_training.jsonl","./finetune_validation.jsonl"], split = "train")

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-32B",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    # token = "hf_...", # use one if using gated models
)
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)

def formatting_prompts_func(examples):
    inputs       = examples["prompt"]
    outputs      = examples["completion"]
    texts = [tokenizer.apply_chat_template([{"role": "user", "content": inputs[idx]},{"role":"assistant","content": outputs[idx]}], tokenize = False, add_generation_prompt = False) for idx in range(0,len(inputs))]
    return { "text" : texts, }
print("Processing data")
dataset = dataset.map(formatting_prompts_func, batched = True,)
# Do model patching and add fast LoRA weights
dataset[1]["text"]

Generating train split: 0 examples [00:00, ? examples/s]

==((====))==  Unsloth 2025.5.6: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/275k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

Processing data


Map:   0%|          | 0/3891 [00:00<?, ? examples/s]

"<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nThe following are multiple choice questions (with answers) about software devopment. There is always only one right answer among choices. Response the final answer with the form The answer is (final choice).\n \nQuestion: What is the use of useState() hook?\n(A) It allows you to have state variables in functional components\n(B) It allows you to have dynamic variables in functional components\n(C) It allows you to create a new state\n(D) None of the above\n\nAnswer: Let's think step by step. <|im_end|>\n<|im_start|>assistant\nThe useState() hook is a fundamental part of React, a popular JavaScript library for building user interfaces. It is specifically used in functional components to add state management capabilities, which were traditionally only available in class components. \n\n- Option (A) states that useState() allows you to have state variables in functional c

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizer),
    dataset_num_proc = 4,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Fixed major bug in latest Unsloth
        warmup_steps = 5,
        num_train_epochs = 2, # Set this for 1 full training run.
        #max_steps = 30,
        learning_rate = 2e-4,
        logging_steps = 20,
        optim = "paged_adamw_8bit", # Save more memory
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc
    ),
)
# trainer = train_on_responses_only(
#     trainer,
#     instruction_part = "<|im_start|>user\n",
#     response_part = "<|im_start|>assistant\n",
# )
tokenizer.decode(trainer.train_dataset[5]["input_ids"])


Unsloth 2025.5.6 patched 64 layers with 64 QKV layers, 64 O layers and 64 MLP layers.


Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/3891 [00:00<?, ? examples/s]

"<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\nThe following are multiple choice questions (with answers) about software devopment. There is always only one right answer among choices. Response the final answer with the form The answer is (final choice).\n \nQuestion: In __________ allocation method for disk block allocation in a file system, insertion and deletion of blocks in a file is easy.\n(A) Index\n(B) Linked\n(C) Contiguous\n(D) Bit Map\n\nAnswer: Let's think step by step. <|im_end|>\n<|im_start|>assistant\nTo determine which disk block allocation method makes insertion and deletion of blocks in a file easy, let's briefly consider each option:\n\n(A) Index: This method uses an index block to keep track of all the blocks of a file. While it provides direct access to any block, insertion and deletion can be complex as it may require updating the index structure.\n\n(B) Linked: In this method, each block conta

In [ ]:
print("Start training")
trainer_stats =trainer.train()

Start training


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,891 | Num Epochs = 2 | Total steps = 972
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 268,435,456/32,000,000,000 (0.84% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
20,0.846900
40,0.424600
60,0.393300
80,0.356700
100,0.352100
120,0.367900
140,0.342000
160,0.358000
180,0.345700
200,0.346200


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
model.save_pretrained("/content/drive/My Drive/finetune_32b")
tokenizer.save_pretrained("/content/drive/My Drive/finetune_32b")

('/content/drive/My Drive/finetune_32b/tokenizer_config.json',
 '/content/drive/My Drive/finetune_32b/special_tokens_map.json',
 '/content/drive/My Drive/finetune_32b/vocab.json',
 '/content/drive/My Drive/finetune_32b/merges.txt',
 '/content/drive/My Drive/finetune_32b/added_tokens.json',
 '/content/drive/My Drive/finetune_32b/tokenizer.json')

In [ ]:
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template
from datasets import load_dataset

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/content/drive/My Drive/finetune_32b", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = 4096,
    dtype = None,
    load_in_4bit = True,
)
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "qwen-2.5",
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
dataset_validate = load_dataset("json", data_files = "./finetune_validation.jsonl",split='train')
messages=[
    {"role": "user", "content": dataset_validate[1]['prompt']},
]
inputs = tokenizer.apply_chat_template(
messages,     tokenize = True, enable_thinking = True,
    add_generation_prompt = True, return_tensors = "pt").to("cuda")
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
print(tokenizer.decode(inputs[0]))
outputs = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 2048, temperature = 0.6, top_p = 0.95, top_k = 20)
#tokenizer.batch_decode(outputs)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.6: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors.index.json:   0%|          | 0.00/275k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

Unsloth 2025.5.6 patched 64 layers with 64 QKV layers, 64 O layers and 64 MLP layers.


Generating train split: 0 examples [00:00, ? examples/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|im_start|>system
You are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>
<|im_start|>user
The following are multiple choice questions (with answers) about software devopment. There is always only one right answer among choices. Response the final answer with the form The answer is (final choice).
 
Question: Look at the problem below, the solution is missing a part, which option is the most likely to complete the solution and achieve the desired goal?

Problem description: You are given an array of strings words. Each element of words consists of two lowercase English letters. Create the longest possible palindrome by selecting some elements from words and concatenating them in any order. Each element can be selected at most once. Return the length of the longest palindrome that you can create. If it is impossible to create any palindrome, return 0. A palindrome is a string that reads the same forward and backward.

Incomplete Solution:
class Solution:
  def lo

In [ ]:
import ast
from string import ascii_uppercase
import pandas as pd

test_df=pd.read_csv('./b6_test_data.csv')
test_df['choices'] = test_df['choices'].apply(lambda x:"\n".join([f"({ascii_uppercase[idx]}) {choice}" for idx, choice in enumerate(ast.literal_eval(x))]))
test_df_questions=test_df['question'].tolist()
test_df_choices=test_df['choices'].tolist()
test_df_taskid=test_df['task_id'].tolist()

In [ ]:
import re
import pickle
import os
import re
import glob

cot_prompt = """The following are multiple choice questions (with answers) about software devopment. There is always only one right answer among choices. Response the final answer with the form The answer is (final choice).

{question}
{choices}

Answer: Let's think step by step. """

def parse_answer(example): #Refernce: https://github.com/FSoft-AI4Code/CodeMMLU/blob/main/src/codemmlu/task_utils.py
        """Answer extract function.

        Args:
            example (str): The example to extract the answer from
        Returns:
            str: The extracted answer
        """
        extract = re.search(r"answer is (\(*[A-E][\).]*)", example, flags=re.IGNORECASE)
        if extract:
            return extract.group(1).replace("(", "").replace(")", "").replace(".", "").strip()


        extract = re.search(r"(\(*[A-E][\).]*) is correct", example, flags=re.IGNORECASE)
        if extract:
            return extract.group(1).replace("(", "").replace(")", "").replace(".", "").strip()
        extract = re.search(r"the correct answer (\(*[A-E][\).]*)", example, flags=re.IGNORECASE)
        if extract:
            return extract.group(1).replace("(", "").replace(")", "").replace(".", "").strip()

        match = re.findall(r"(A|B|C|D|E)", example)

        if match:
            # if len(match) > 1:
            #     return None
            return list(match)[0] # Take the first one
        return None


def get_response(model,question,choices,prompt):
    messages=[
        {"role": "user", "content": prompt.format(question=question,choices=choices)},
    ]
    inputs = tokenizer.apply_chat_template(messages,tokenize = True,enable_thinking = True,
                                           add_generation_prompt = True, return_tensors = "pt").to("cuda")
    outputs = model.generate(input_ids = inputs, max_new_tokens = 2048, temperature = 0.6, top_p = 0.95, top_k = 20)

    response=tokenizer.batch_decode(outputs)[0]
    answer=parse_answer(response)
    assert answer!=None, response
    return answer,response

finished_index=[]

for file_path in glob.iglob(f'/content/drive/My Drive/finetune_32b/response_submission/*.pkl'):
    file_name=os.path.basename(file_path)
    index=file_name.split('.')[0]
    finished_index.append(index)

result_taskid=[]
result_answer=[]
for idx in range(len(test_df_questions)):
    if idx%50==0:
        print("Processing ",idx)
    if test_df_taskid[idx] in finished_index:
      open_file = open(f"/content/drive/My Drive/finetune_32b/response_submission/{test_df_taskid[idx]}.pkl", 'rb')
      response_data=pickle.load(open_file)
      result_taskid.append(test_df_taskid[idx])
      result_answer.append(response_data['answer'])
      continue
    result_taskid.append(test_df_taskid[idx])
    answer,response=get_response(model,test_df_questions[idx],test_df_choices[idx],cot_prompt)
    result_answer.append(answer.strip())
    dict_to_save={'answer':answer.strip(),'response':response}
    save_file = open(f"/content/drive/My Drive/finetune_32b/response_submission/{test_df_taskid[idx]}.pkl", 'wb')
    pickle.dump(dict_to_save, save_file)
    save_file.close()
print(f"Finish !!!")
dict_for_submission={'task_id':result_taskid,'answer':result_answer}
submission_df=pd.DataFrame(dict_for_submission)
submission_df.to_csv('/content/drive/My Drive/finetune_32b/submission.csv', index=False)

Processing  0
Processing  50
Processing  100
Processing  150
Processing  200
Processing  250
Processing  300
Processing  350
Processing  400
Processing  450
Processing  500
Processing  550
Processing  600
Processing  650
Processing  700
Processing  750
Processing  800
Processing  850
Processing  900
Processing  950
Processing  1000
Processing  1050
Processing  1100
Processing  1150
Processing  1200
Processing  1250
Finish !!!


### Baseline result

In [ ]:
from getpass import getpass
OpenRouter_API_KEY=getpass("OpenRouter API Key: ")

OpenRouter API Key: ··········


In [ ]:
import re
import pickle
import os
import re
import glob
import requests
import json
import os

cot_prompt = """The following are multiple choice questions (with answers) about software devopment. There is always only one right answer among choices. Response the final answer with the form The answer is (final choice).

{question}
{choices}

Answer: Let's think step by step. """

def parse_answer(example): #Refernce: https://github.com/FSoft-AI4Code/CodeMMLU/blob/main/src/codemmlu/task_utils.py
        """Answer extract function.

        Args:
            example (str): The example to extract the answer from
        Returns:
            str: The extracted answer
        """
        extract = re.search(r"answer is (\(*[A-E][\).]*)", example, flags=re.IGNORECASE)
        if extract:
            return extract.group(1).replace("(", "").replace(")", "").replace(".", "").strip()


        extract = re.search(r"(\(*[A-E][\).]*) is correct", example, flags=re.IGNORECASE)
        if extract:
            return extract.group(1).replace("(", "").replace(")", "").replace(".", "").strip()
        extract = re.search(r"the correct answer (\(*[A-E][\).]*)", example, flags=re.IGNORECASE)
        if extract:
            return extract.group(1).replace("(", "").replace(")", "").replace(".", "").strip()

        match = re.findall(r"(A|B|C|D|E)", example)

        if match:
            # if len(match) > 1:
            #     return None
            return list(match)[0] # Take the first one
        return None


url = "https://openrouter.ai/api/v1/chat/completions"
headers = {
  "Authorization": f"Bearer {OpenRouter_API_KEY}",
  "Content-Type": "application/json"
}

def get_response(question,choices,prompt):
    payload = {
      "model": "qwen/qwen3-32b",
      "messages": [
        {
          "role": "user",
          "content": prompt.format(question=question,choices=choices)
        }
      ],
      "temperature": 0.6,
      "top_k": 20,
      "top_p": 0.9,
      'provider': {
        'sort': 'throughput'
      }
    }
    response_request= requests.post(url, headers=headers, json=payload)
    response=response_request.json()['choices'][0]['message']['content']
    answer=parse_answer(response)
    assert answer!=None, response
    return answer,response

finished_index=[]

for file_path in glob.iglob(f'/content/drive/My Drive/finetune_32b/response_submission_baseline_qwen/*.pkl'):
    file_name=os.path.basename(file_path)
    index=file_name.split('.')[0]
    finished_index.append(index)

result_taskid=[]
result_answer=[]
for idx in range(len(test_df_questions)):
    if idx%50==0:
        print("Processing ",idx)
    if test_df_taskid[idx] in finished_index:
      open_file = open(f"/content/drive/My Drive/finetune_32b/response_submission_baseline_qwen/{test_df_taskid[idx]}.pkl", 'rb')
      response_data=pickle.load(open_file)
      result_taskid.append(test_df_taskid[idx])
      result_answer.append(response_data['answer'])
      continue
    result_taskid.append(test_df_taskid[idx])
    answer,response=get_response(test_df_questions[idx],test_df_choices[idx],cot_prompt)
    result_answer.append(answer.strip())
    dict_to_save={'answer':answer.strip(),'response':response}
    save_file = open(f"/content/drive/My Drive/finetune_32b/response_submission_baseline_qwen/{test_df_taskid[idx]}.pkl", 'wb')
    pickle.dump(dict_to_save, save_file)
    save_file.close()
print(f"Finish baseline Qwen!!!")
dict_for_submission={'task_id':result_taskid,'answer':result_answer}
submission_df=pd.DataFrame(dict_for_submission)
submission_df.to_csv('/content/drive/My Drive/finetune_32b/submission-baseline-qwen.csv', index=False)

Processing  0
Processing  50
Processing  100
Processing  150
Processing  200
Processing  250
Processing  300
Processing  350
Processing  400
Processing  450
Processing  500
Processing  550
Processing  600
Processing  650
Processing  700
Processing  750
Processing  800
Processing  850
Processing  900
Processing  950
Processing  1000
Processing  1050
Processing  1100
Processing  1150
Processing  1200
Processing  1250
Finish baseline Qwen!!!
